In [1]:
pip install openai

Note: you may need to restart the kernel to use updated packages.


In [2]:
#completeness
import pandas as pd
import time
from openai import OpenAI

client = OpenAI(
    api_key="sk-0c4e0db9a2af4fad8f3bc80d63b6c8ed", 
    base_url="https://api.deepseek.com/v1"
)

df = pd.read_csv("eval_manual_check_dataset.csv")

prompt_template = """
You are an expert evaluator. Your task is to judge how completely a model's answer captures the key ideas of an ideal answer to a given question.

You will be shown:
- The original question
- An ideal reference answer
- A model-generated answer

Your goal is to evaluate how well the model answer includes the core ideas of the ideal answer.

Scoring Criteria:
- 0 = The model's answer misses the point completely (none of the main ideas are present)
- 1 = The model's answer includes some but not all of the main ideas
- 2 = The model's answer includes all the main ideas clearly and accurately

Respond in this exact format:
Completeness: X

Only respond with the score number. Do not add explanations or comments.

Question:
{question}

Ideal Answer:
{ideal_answer}

Model Answer:
{model_answer}
"""

scores = []

for _, row in df.iterrows():
    filled_prompt = prompt_template.format(
        question=row["Question"],
        ideal_answer=row["Ideal Answer"],
        model_answer=row["Model Answer"]
    )

    try:
        response = client.chat.completions.create(
            model="deepseek-chat",
            messages=[
                {"role": "system", "content": "You are an expert evaluator."},
                {"role": "user", "content": filled_prompt}
            ],
            temperature=0
        )

        reply = response.choices[0].message.content.strip()
        if "Completeness:" in reply:
            score = reply.split("Completeness:")[1].strip().split()[0]
        else:
            score = ""

    except Exception as e:
        print(f"Error: {e}")
        score = ""

    scores.append(score)
    time.sleep(1)

df["completeness"] = scores
df.to_csv("eval_with_completeness_deepseekchat.csv", index=False)


In [11]:
#rebundancy_ratio
import pandas as pd
import time
from openai import OpenAI

client = OpenAI(
    api_key="sk-0c4e0db9a2af4fad8f3bc80d63b6c8ed", 
    base_url="https://api.deepseek.com/v1"
)

df = pd.read_csv("eval_manual_check_dataset.csv")

prompt_template = """
You are an expert evaluator. Your task is to judge the Redundancy Ratio of a model-generated answer by comparing it to an ideal reference answer.

Your goal is to determine whether the model included more information than what was expected based on the ideal answer.

Scoring Criteria:
- 0 = The model gives exactly the information expected — no more, no less.
- 1 = The model adds a little more than what was expected, but it's acceptable.
- 2 = The model adds significantly more information than desired.

Respond in this exact format:
Redundancy Ratio: X

Only respond with the score number. Do not add explanations or comments.

Be strict and concise. Do not explain. Do not return more than one line.

IMPORTANT: Only respond with "Redundancy Severity: X" — where X is 0, 1, or 2.

Question:
{question}

Ideal Answer:
{ideal_answer}

Model Answer:
{model_answer}
"""

scores = []

for _, row in df.iterrows():
    filled_prompt = prompt_template.format(
        question=row["Question"],
        ideal_answer=row["Ideal Answer"],
        model_answer=row["Model Answer"]
    )

    try:
        response = client.chat.completions.create(
            model="deepseek-chat",
            messages=[
                {"role": "system", "content": "You are an expert evaluator."},
                {"role": "user", "content": filled_prompt}
            ],
            temperature=0
        )

        reply = response.choices[0].message.content.strip()
        if "Redundancy Ratio:" in reply:
            score = reply.split("Redundancy Ratio:")[1].strip().split()[0]
        else:
            score = ""

    except Exception as e:
        print(f"Error: {e}")
        score = ""

    scores.append(score)
    time.sleep(1)

df["redundancy_ratio"] = scores
df.to_csv("eval_with_rebundancy_ratio_deepseekchat.csv", index=False)


In [10]:
#rebundancy severity
import pandas as pd
import time
from openai import OpenAI

client = OpenAI(
    api_key="sk-0c4e0db9a2af4fad8f3bc80d63b6c8ed", 
    base_url="https://api.deepseek.com/v1"
)

df = pd.read_csv("eval_manual_check_dataset.csv")

prompt_template = """
You are an expert evaluator. Your task is to judge the Redundancy Severity of a model-generated answer by comparing it to an ideal reference answer.

Your goal is to evaluate the **impact** of any extra or redundant content in the model's answer. Focus on how distracting, confusing, or misleading it is compared to the expected answer.

Scoring Criteria:
- 0 = No redundancy. The answer is focused and accurate.
- 1 = Minor distraction. There is some unnecessary information, but it does not confuse the meaning.
- 2 = Major misunderstanding. The extra content significantly distracts, misleads, or alters the expected meaning.

Respond in this exact format:
Redundancy Severity: X

Only respond with the score number. Do not add explanations or comments.

Be strict and concise. Do not explain. Do not return more than one line.

IMPORTANT: Only respond with "Redundancy Severity: X" — where X is 0, 1, or 2.

Question:
{question}

Ideal Answer:
{ideal_answer}

Model Answer:
{model_answer}

"""

scores = []

for _, row in df.iterrows():
    filled_prompt = prompt_template.format(
        question=row["Question"],
        ideal_answer=row["Ideal Answer"],
        model_answer=row["Model Answer"]
    )

    try:
        response = client.chat.completions.create(
            model="deepseek-chat",
            messages=[
                {"role": "system", "content": "You are an expert evaluator."},
                {"role": "user", "content": filled_prompt}
            ],
            temperature=0
        )

        reply = response.choices[0].message.content.strip()
        if "Redundancy Severity:" in reply:
            score = reply.split("Redundancy Severity:")[1].strip().split()[0]
        else:
            score = ""

    except Exception as e:
        print(f"Error: {e}")
        score = ""

    scores.append(score)
    time.sleep(1)

df["redundancy_severity"] = scores
df.to_csv("eval_with_rebundancy_severity_deepseekchat.csv", index=False)
